In [ ]:
# Ensure repo and src/ are importable and .env is loaded
import notebooks._bootstrap_src_path  # noqa: F401


# Indexing

This part indexes documents (creates embedding and stores them in a Haystack DocumentStore). It uses different indexing variant s.a. normal indexing and contextualized indexing (used later for contextual RAG evaluation).

In [ ]:
# Dependencies are managed via the repository's virtual environment.
# Install once from the project root using `requirements.txt`.
# This notebook does not run `%pip install` commands.

In [1]:
import os
from utils.markdown_utils import for_each_markdown_file
import pandas as pd
from haystack.document_stores.in_memory import InMemoryDocumentStore
from tqdm import tqdm
tqdm.pandas()

from pipelines.indexing_pipelines.base_indexing_pipeline import get_base_indexing_pipeline
from pipelines.indexing_pipelines.context_indexing_pipeline import get_context_indexing_pipeline
from models import EmbeddingModelConfig, EmbeddingModelProvider, LLMProvider, LLMConfig

import logging
from utils.pickle_utils import for_each_pickle_file

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s',
    datefmt='%H:%M:%S'
)

# silence haystack’s pipeline logs
logging.getLogger("haystack").setLevel(logging.WARNING)
logging.getLogger("haystack.core.pipeline").setLevel(logging.WARNING)

# if you see similar spam from transformers, ragas, etc.
logging.getLogger("transformers").setLevel(logging.WARNING)
logging.getLogger("ragas").setLevel(logging.WARNING)

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env (bootstrap also does this; safe to call again)
load_dotenv()

# Set defaults for caches/context size if not already provided
os.environ.setdefault("SENTENCE_TRANSFORMERS_HOME", "./model-assets/sentence-transformers")
os.environ.setdefault("LLM_CONTEXT_SIZE", "40000")

embedding_model_name = "Qwen/Qwen3-Embedding-8B"
embedding_model_provider = EmbeddingModelProvider.SENTENCE_TRANSFORMER

contexualization_model_name = "gpt-5-mini-2025-08-07"
contexualization_model_provider = LLMProvider.OPEN_AI

Check maximum passage length so make sure every chunk fits in the context.

In [ ]:
df = pd.read_pickle("data/preprocessed_documents/docs_passage_1_0.pkl")

doc_contents = df["document"].map(lambda doc: doc.content)
doc_contents.apply(len).max()

np.int64(1460)

## Base Indexing

In [3]:
def base_indexing(filename, df):
    documents = df["document"].tolist()

    base_indexing_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
    base_indexing_pipeline = get_base_indexing_pipeline(
        base_indexing_store, 
        embedding_model_config=EmbeddingModelConfig(name=embedding_model_name, provider=EmbeddingModelProvider.SENTENCE_TRANSFORMER),
    )

    def index_documents(filename, bytes):
        documents_from_file = [document for document in documents if document.meta["title"] == filename]
        base_indexing_pipeline.run({
            "embedder": {
                "documents": documents_from_file
            }
        })

    for_each_markdown_file("data/md_files", index_documents)

    filepath = f"data/document_stores/{embedding_model_name}/base"
    os.makedirs(filepath, exist_ok=True)
    clean_name = os.path.splitext(os.path.basename(filename))[0]
    base_indexing_store.save_to_disk(f"{filepath}/{clean_name}_indexing_store.json")

for_each_pickle_file("data/preprocessed_documents", base_indexing)

Processing Pickle files:   0%|          | 0/8 [00:00<?, ?it/s]16:12:18 INFO Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-8B


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

16:14:10 INFO 2 prompts are loaded, with the keys: ['query', 'document']


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Pickle files: 100%|██████████| 8/8 [04:24<00:00, 33.11s/it]


## Context Indexing

In [ ]:
def context_indexing(filename, df):
    documents = df["document"].tolist()

    context_indexing_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
    context_indexing_pipeline = get_context_indexing_pipeline(
        context_indexing_store, 
        embedding_model_config=EmbeddingModelConfig(name=embedding_model_name, provider=EmbeddingModelProvider.SENTENCE_TRANSFORMER), 
        contextualizer_model_config=LLMConfig(name=contexualization_model_name, provider=contexualization_model_provider)
    )

    def index_with_context(filename, bytes):
        documents_from_file = [document for document in documents if document.meta["title"] == filename]
        file_content = bytes.decode("utf-8")
        context_indexing_pipeline.run({
            "contextualiser": {
                "context": file_content,
                "documents": documents_from_file
            }
        })

    for_each_markdown_file("data/md_files", index_with_context)

    filepath = f"data/document_stores/{embedding_model_name}/context/{contexualization_model_name}"
    os.makedirs(filepath, exist_ok=True)
    clean_name = os.path.splitext(os.path.basename(filename))[0]
    context_indexing_store.save_to_disk(f"{filepath}/{clean_name}_indexing_store.json")

for_each_pickle_file("data/preprocessed_documents", context_indexing)

Processing Pickle files:   0%|          | 0/6 [00:00<?, ?it/s]21:23:54 INFO Load pretrained SentenceTransformer: Qwen/Qwen3-Embedding-8B


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

21:24:01 INFO 2 prompts are loaded, with the keys: ['query', 'document']
21:24:07 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
21:24:13 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
21:24:18 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
21:24:23 INFO HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Processing Pickle files:   0%|          | 0/6 [00:31<?, ?it/s]


KeyboardInterrupt: 

#### Index already contextualized documents

Use this if only the embedding model changes, but you don't want to re-contextualize all the documents

In [ ]:
# from utils.json_utils import for_each_document_store
# from pipelines.indexing_pipelines.base_indexing_pipeline import get_base_indexing_pipeline
# from models import EmbeddingModelProvider, EmbeddingModelConfig
# from haystack.document_stores.in_memory import InMemoryDocumentStore
# from config.secret import OPENAI_API_KEY
# import os

# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# contextualizer_model_name = "gemma3:27b"

# old_embedding_model_name = "Qwen/Qwen3-Embedding-8B"

# new_embedding_model_name = "text-embedding-3-large"
# new_embedding_model_provider = EmbeddingModelProvider.OPENAI

# def embeddings_for_contextualized_chunks(filename, old_store):
#     contextualized_documents = old_store.filter_documents()

#     context_indexing_store = InMemoryDocumentStore(embedding_similarity_function="cosine")
#     base_indexing_pipeline = get_base_indexing_pipeline(
#         context_indexing_store, 
#         EmbeddingModelConfig(new_embedding_model_name, new_embedding_model_provider)
#     )
#     base_indexing_pipeline.run({
#         "embedder": { 
#             "documents": contextualized_documents
#         },
#     })

#     filepath = f"data/document_stores/{new_embedding_model_name}/context/{contextualizer_model_name}"
#     os.makedirs(filepath, exist_ok=True)
#     context_indexing_store.save_to_disk(f"{filepath}/{filename}")


# for_each_document_store(f"data/document_stores/{old_embedding_model_name}/context/{contextualizer_model_name}", embeddings_for_contextualized_chunks)

Calculating embeddings: 13it [00:08,  1.61it/s] | 0/6 [00:00<?, ?it/s]
Calculating embeddings: 10it [00:08,  1.24it/s] | 1/6 [00:11<00:56, 11.27s/it]
Calculating embeddings: 5it [00:04,  1.20it/s]  | 2/6 [00:21<00:43, 10.86s/it]
Calculating embeddings: 2it [00:01,  1.15it/s]  | 3/6 [00:27<00:25,  8.38s/it]
Calculating embeddings: 10it [00:19,  1.96s/it] | 4/6 [00:29<00:11,  5.90s/it]
Calculating embeddings: 1it [00:01,  1.12s/it]▎ | 5/6 [00:51<00:11, 11.73s/it]
Processing document store files: 100%|██████████| 6/6 [00:52<00:00,  8.80s/it]


## Validation

In [ ]:
# base_documents = base_indexing_store.filter_documents()
base_documents = InMemoryDocumentStore.load_from_disk("data/document_stores/Qwen/Qwen3-Embedding-4B/base/docs_passage_1_0_indexing_store.json").filter_documents()
contextualized_documents = InMemoryDocumentStore.load_from_disk("data/document_stores/Qwen/Qwen3-Embedding-4B/context/gpt-4.1-mini/docs_passage_1_0_indexing_store.json").filter_documents()

In [ ]:
import numpy as np

# Get content lengths
base_lengths = [len(doc.content) for doc in base_documents]
contextualized_lengths = [len(doc.content) for doc in contextualized_documents]

# Compute stats
base_mean = np.mean(base_lengths)
base_std = np.std(base_lengths)

contextualized_mean = np.mean(contextualized_lengths)
contextualized_std = np.std(contextualized_lengths)

print(f"Base documents - Mean: {base_mean:.2f} chars, Std Dev: {base_std:.2f}")
print(f"Contextualized documents - Mean: {contextualized_mean:.2f} chars, Std Dev: {contextualized_std:.2f}")

Base documents - Mean: 317.14 chars, Std Dev: 179.72
Contextualized documents - Mean: 606.93 chars, Std Dev: 191.06


In [ ]:
[document.content for document in contextualized_documents]

['Ein Kaiserschnitt ist ein chirurgischer Eingriff zur Entbindung des Kindes, der entweder geplant oder akut notwendig sein kann. Die Dringlichkeit richtet sich nach der Situation von Mutter und Kind, um Risiken während der Geburt zu minimieren.\n\nEin Kaiserschnitt kann aus unterschiedlichen Gründen und mit variierender Dringlichkeit notwendig werden. Er kann im Voraus geplant sein, etwa wenn Sie bereits mehrere Kaiserschnitte hatten, das Kind sich in Querlage befindet oder bestimmte Erkrankungen vorliegen. In anderen Fällen ergibt sich der Bedarf erst während der laufenden Geburt – etwa bei Geburtsstillstand oder wenn Mutter oder Kind gefährdet sind. Dabei unterscheidet man:\n\n',
 'Ein Kaiserschnitt wird je nach Dringlichkeit in drei Kategorien eingeteilt: geplant ohne akuten Handlungsbedarf, dringlich bei beginnender Gefährdung von Mutter oder Kind, und Notfall, wenn sofortiges Handeln lebensrettend ist. Die Entscheidung basiert auf der aktuellen Situation von Mutter und Kind währe